## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
from citipy import citipy
import random
import time
from datetime import datetime 
from config import weather_api_key
from scipy.stats import linregress
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_weather_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_weather_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,Longyearbyen,78.2186,15.6401,42.64,87,75,17.27,IN,broken clouds
1,1,Kaolack,14.1825,-16.2533,84.49,61,53,12.01,IN,broken clouds
2,2,Adrar,20.5022,-10.0711,93.63,25,63,5.48,IN,broken clouds
3,3,Jamestown,42.0970,-79.2353,73.71,84,75,5.01,IN,broken clouds
4,4,Ushuaia,-54.8000,-68.3000,37.06,75,75,5.75,IN,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_weather_df = city_weather_data_df.loc[(city_weather_data_df["Max Temp"] <= max_temp) & \
                                       (city_weather_data_df["Max Temp"] >= min_temp)]
preferred_weather_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
1,1,Kaolack,14.1825,-16.2533,84.49,61,53,12.01,IN,broken clouds
6,6,Pemberton,50.3164,-122.8027,87.49,34,26,6.87,IN,scattered clouds
11,11,Praia Da Vitoria,38.7333,-27.0667,75.45,78,20,14.65,IN,few clouds
20,20,Albany,42.6001,-73.9662,80.02,58,90,1.99,IN,overcast clouds
22,22,Santa Cruz,-17.8000,-63.1667,82.45,47,0,19.57,IN,clear sky
23,23,Iwaki,37.0500,140.8833,81.01,76,3,1.01,IN,clear sky
24,24,Kavaratti,10.5669,72.6420,82.08,79,84,19.57,IN,broken clouds
26,26,Ransang,8.9603,117.5828,81.37,79,100,9.91,IN,overcast clouds
29,29,Hithadhoo,-0.6000,73.0833,81.75,78,48,10.78,IN,scattered clouds
36,36,Atuona,-9.8000,-139.0333,78.67,71,36,17.49,IN,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_weather_df.count()

City_ID                244
City                   244
Lat                    244
Lng                    244
Max Temp               244
Humidity               244
Cloudiness             244
Wind Speed             244
Country                244
Weather Description    244
dtype: int64

In [6]:
preferred_weather_df.isnull()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
1,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False,False,False,False
20,False,False,False,False,False,False,False,False,False,False
22,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
691,False,False,False,False,False,False,False,False,False,False
693,False,False,False,False,False,False,False,False,False,False
696,False,False,False,False,False,False,False,False,False,False
697,False,False,False,False,False,False,False,False,False,False


In [7]:
preferred_weather_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Weather Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
travel_cities_df=preferred_weather_df.dropna()
travel_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
1,1,Kaolack,14.1825,-16.2533,84.49,61,53,12.01,IN,broken clouds
6,6,Pemberton,50.3164,-122.8027,87.49,34,26,6.87,IN,scattered clouds
11,11,Praia Da Vitoria,38.7333,-27.0667,75.45,78,20,14.65,IN,few clouds
20,20,Albany,42.6001,-73.9662,80.02,58,90,1.99,IN,overcast clouds
22,22,Santa Cruz,-17.8000,-63.1667,82.45,47,0,19.57,IN,clear sky
...,...,...,...,...,...,...,...,...,...,...
691,691,Hobyo,5.3505,48.5268,76.39,78,71,25.66,IN,broken clouds
693,693,Bastia,42.6667,9.3333,78.60,91,90,5.75,IN,overcast clouds
696,696,San Felipe,10.3399,-68.7425,77.02,83,100,2.51,IN,overcast clouds
697,697,Palatka,29.6486,-81.6376,87.35,67,47,9.04,IN,scattered clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = travel_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kaolack,IN,84.49,broken clouds,14.1825,-16.2533,
6,Pemberton,IN,87.49,scattered clouds,50.3164,-122.8027,
11,Praia Da Vitoria,IN,75.45,few clouds,38.7333,-27.0667,
20,Albany,IN,80.02,overcast clouds,42.6001,-73.9662,
22,Santa Cruz,IN,82.45,clear sky,-17.8000,-63.1667,
23,Iwaki,IN,81.01,clear sky,37.0500,140.8833,
24,Kavaratti,IN,82.08,broken clouds,10.5669,72.6420,
26,Ransang,IN,81.37,overcast clouds,8.9603,117.5828,
29,Hithadhoo,IN,81.75,scattered clouds,-0.6000,73.0833,
36,Atuona,IN,78.67,scattered clouds,-9.8000,-139.0333,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat= row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    
     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
     #6e. Make request and retrieve the JSON data from the search. 
    hotel_location = requests.get(base_url, params=params).json()
                     
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: hotel_df.loc[index, "Hotel Name"] = hotel_location["results"][0]["name"]
    
    except (IndexError):
        print("Cannot find hotel...skipping")

Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping
Cannot find hotel...skipping


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Kaolack,IN,84.49,broken clouds,14.1825,-16.2533,
6,Pemberton,IN,87.49,scattered clouds,50.3164,-122.8027,Pemberton Valley Lodge
11,Praia Da Vitoria,IN,75.45,few clouds,38.7333,-27.0667,Hotel Branco I
20,Albany,IN,80.02,overcast clouds,42.6001,-73.9662,
22,Santa Cruz,IN,82.45,clear sky,-17.8000,-63.1667,Aviador Hotel Boutique


In [18]:
# 8a. Create the output File (CSV)
output_hotel_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_hotel_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
1,14.1825,-16.2533
6,50.3164,-122.8027
11,38.7333,-27.0667
20,42.6001,-73.9662
22,-17.8000,-63.1667
...,...,...
691,5.3505,48.5268
693,42.6667,9.3333
696,10.3399,-68.7425
697,29.6486,-81.6376


In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))